In [1]:
import pandas as pd
from pricer.pricer import Pricer
from pricer.pricer_factory import PricerFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
## initializing pricer_class
pricer_class = PricerFactory.build(Pricer.DAILY_STOCK_ROLLING)

In [3]:
backtest_start_date = datetime(2001,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [4]:
## creating database objects
market = Market()
sec = SEC()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [8]:
market.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = pricer_class.training_set(ticker,prices,False)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [01:07<00:00,  7.24it/s]


In [9]:
data = pd.concat(training_sets)

In [11]:
training_data = data.dropna().copy().sort_values(["year","date"])

In [12]:
training_data = training_data.groupby(["year","quarter","date","ticker"]).mean().reset_index()

In [13]:
training_data

,year,quarter,date,ticker,price_prediction
0,2000,2,2000-05-24,A,59.717194
1,2000,2,2000-05-24,AAPL,0.871308
2,2000,2,2000-05-24,ABC,3.326919
3,2000,2,2000-05-24,ABMD,24.289600
4,2000,2,2000-05-24,ABT,8.842085
...,...,...,...,...,...
2537671,2023,3,2023-07-21,YUM,133.652339
2537672,2023,3,2023-07-21,ZBH,134.108355
2537673,2023,3,2023-07-21,ZBRA,287.979500
2537674,2023,3,2023-07-21,ZION,30.331782


In [14]:
pricer_class.db.connect()
relevant_columns = list(set(list(training_data.columns)) - set(pricer_class.factors))
pricer_class.db.store("sim",training_data[relevant_columns])
pricer_class.db.disconnect()

In [ ]:
pricer_class.db.connect()
pricer_class.db.drop("predictions")
relevant_columns = list(set(list(training_data.columns)) - set(pricer_class.factors))
predictions = pricer_class.sim_processor(training_data)
pricer_class.db.store("predictions",predictions[predictions["year"]==current_year][relevant_columns])
pricer_class.db.disconnect()